<div align="center">
    <img src='https://media-exp1.licdn.com/dms/image/C561BAQFV4oU27rzxaQ/company-background_10000/0/1562869489836?e=2147483647&v=beta&t=OMHL6Izxqw7r8S5fn9gDQM1hNMKgo3yqMlH1F3Iwm9o' style='height:300px;'>
</div>


<center><h1 style='font-size:35px'>Análisis y visualización de datos - Encuesta Sysarmy</h1></center>

---

**Fecha de creación: 15/09/2022**

**Grupo: 6**

**Autores**:
* Marilina Trevisan (marilinatrevisan@gmail.com).
* Ani Salama (anisalama@gmail.com).
* Anahí Sulca (nanisulca@gmail.com).
* Gustavo Ariel Venchiarutti (gustavo.venchiarutti@gmail.com).
* Agustín Trulli (agustintrulli@gmail.com).

#Consigna

Práctico entregable: 
Utilizar la base de jugadores “players_22.csv” disponible en la página de Kaggle https://www.kaggle.com/datasets/stefanoleone992/fifa-22-complete-player-dataset . Considerar que la base 2022 no tiene el mismo formato que la base vista en clase, a los nombres de las variables se les agregó una keyword para identificar a qué tipo de habilidad corresponde.

Con la nueva base, realizar un análisis análogo al que realizamos en el cursado de la materia con los datos FIFA2019. Realice comentarios en cada parte (verbose=True ;))

1. Análisis exploratorio de la base.
2. Evaluación visual  e intuitiva de a dos variables numéricas por vez.
3. Uso de dos técnicas de clustering: por ejemplo k-medias, DBSCAN, mezcla de Gaussianas y/o alguna jerárquica. Elección justificada de hiper-parámetros
4. Evaluación y Análisis de los clusters encontrados.
5. Pregunta: ¿Se realizó alguna normalización o escalado de la base? ¿Por qué ?
6. Uso de alguna transformación (proyección, Embedding) para visualizar los resultados y/o usarla como preprocesado para aplicar alguna técnica de clustering.


## 1 - Inicialización del entorno

**Empezamos cargando algunas herramientas para cargar los datos y manipularlos.**

In [ ]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',1000)
import itertools
import warnings
warnings.filterwarnings("ignore")
import io

**Para visualización usaremos principalmente plotly, también seaborn y matplotlib.**

In [ ]:
from plotly.offline import init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
import matplotlib.pyplot as plt
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
import seaborn as sns
from sklearn.manifold import TSNE
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

**Liberias de metodos de CLUSTERING.**

In [ ]:
from sklearn.cluster import KMeans, MeanShift, DBSCAN
from sklearn.preprocessing import normalize, StandardScaler
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture

**Librerias para definicion de Metricas**

In [ ]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix

## 2 - Exploración de la base

In [ ]:
# Reading data
ROOT_PATH = os.path.dirname(os.getcwd())
DATA_PATH = os.path.join(ROOT_PATH, 'data', 'raw')

df = pd.read_csv(os.path.join(DATA_PATH, 'players_22.csv'))

In [ ]:
print('Cantidad de variables para analizar', len(df.columns))

In [ ]:
df.columns.tolist()

In [ ]:
df.isna().sum()

In [ ]:
# Analisis de parametros estadisticos en todas las variables numericas
print(df.describe())

### Analisis de Variables

#### PLAYER_POSITIONS

In [ ]:
best_players_per_position = df.iloc[df.groupby(df['player_positions'])['overall'].idxmax()][['player_positions','short_name','overall']]
best_players_per_position.head()

In [ ]:
best_players_per_position.info()

Como podemos ver para cada jugador en el mismo campo de posisiones podemos tener hasta 3 valores, lo cual al analizar los mejores jugadores por posicion me genera un data frame de 674 registros.

Por lo tanto optamos por tomar como principal la primera posicion que se encuentra en esta campo, el resto se descarto.

In [ ]:
# Transformo en un string separado por , al campo de posiciones que tiene mas de una alternativa
df['player_positions'] = df['player_positions'].map(lambda x: str(x))

# Transformado el campo ahora tomo la primera posicion que figura en las lista de alternativas
df['player_positions_principal'] = df['player_positions'].apply(lambda x: x.split(",")[0] if x != 'nan' else None)

In [ ]:
# Candidad de jugadores por player_positions_principal
pd.DataFrame(df.player_positions_principal.value_counts().sort_index())

In [ ]:
# Repertimos el analisis anterior y en funcion de la prosicion que consideramos como principal 
# sacamos los mejores jugadores segun el overall definido
best_players_per_position = df.iloc[df.groupby(df['player_positions_principal'])['overall'].idxmax()][['player_positions', 'player_positions_principal','short_name','overall']]
best_players_per_position

In [ ]:
# Clubes que tienen mejor promedio de Overall entre sus jugadores
club_avg_overall = df.groupby("club_name")["overall"].mean().reset_index().sort_values("overall",ascending=False)
club_avg_overall.head(10)

#### OVERALL

La variable **'Overall'** toma valores enteros entre 0 y 100, se calcula usando otras variables de desempeño del jugador (skills_ratings), utilizando redondeo. Por lo tanto la variable 'Overall' figura como numérica discreta, pero como se redondea es de "naturaleza" continua. 

In [ ]:
# Realizamos el histograma de la variable "Overall" (Desempeño global)
nbins = 1*(df.overall.max()-df.overall.min())+1
df['overall'].hist(bins = nbins)
print(nbins)

#### OTRAS VARIABLES
Las demas variables que definen el desempeño (overall) del jugador segun la habilidad que cada una represnta ('attacking_crossing', 'attacking_finishing',
'skill_dribbling', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_agility', 'power_shot_power',  'power_jumping', etc) toman valores enteros entre 0 y 100 tambien redondeados y que juntos permiten obtener el desempeño total del jugador (overall)

In [ ]:
# Realizamos el histograma de la variable "attacking_crossing" (Desempeño global)
nbins = 1*(df.attacking_crossing.max()-df.attacking_crossing.min())+1
df['attacking_crossing'].hist(bins = nbins)
print(nbins)

In [ ]:
# Realizamos el histograma de la variable "skill_ball_control" (Desempeño global)
nbins = 1*(df.skill_ball_control.max()-df.skill_ball_control.min())+1
df['skill_ball_control'].hist(bins = nbins)
print(nbins)

In [ ]:
# Realizamos el histograma de la variable "power_shot_power" (Desempeño global)
nbins = 1*(df.power_shot_power.max()-df.power_shot_power.min())+1
df['power_shot_power'].hist(bins = nbins)
print(nbins)

## 3 - Preparación-de-los-datos-a analizar

### Data Frame solo de Mejores Jugadores

In [ ]:
# Realizo una copia del data frame para empezar el analisis
df_fifa22 = df.copy()

In [ ]:
# Definimos una cantidad de futbolistas para analizar
n=10000

# Reducimos el data set a la cantidad de analisis
df_fifa22 = df.loc[:n]

# De la cantidad que definimos solo se tomaron los jugadores que superan el 70% de overall
df_fifa22 = df_fifa22[(df_fifa22['overall']>70)]

df_fifa22

### Variables de desempeño

Se definen variables generales para todos, en el caso de los jugadores solamente, las columnas de datos que referencian el desempeños de arqueros, **'goalkeeping_diving'**, **'goalkeeping_handling'**, **'goalkeeping_kicking'**, **'goalkeeping_positioning'**,** 'goalkeeping_reflexes'** tendran valor 0

In [ ]:
skills_ratings = ['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing',
                  'attacking_volleys', 'skill_dribbling','skill_curve', 'skill_fk_accuracy',  'skill_long_passing', 'skill_ball_control', 'movement_acceleration',
                  'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength',
                  'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties',
                  'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_diving', 'goalkeeping_handling',
                  'goalkeeping_kicking', 'goalkeeping_positioning', 'goalkeeping_reflexes']
print(len(skills_ratings), 'variables numéricas de desempeño según habilidad')

### Jugador Perfecto

In [ ]:
# Ponemos al overall el valor 99 
MachineGunDict = {'short_name':'MachineGun','overall':99}

# Para todas las columnas de skils en este jugador ponemos el valor 99 y lo agregamos a la tabla general
for skills in skills_ratings:
    MachineGunDict[skills]=99
    
df_fifa22 = df_fifa22.append(MachineGunDict,ignore_index=True)

In [ ]:
# Genero un data frame, sólo con desempeños según habilidad (numéricas)
df_skills = df_fifa22[skills_ratings]
df_skills

### Division estrategica de posiciones

In [ ]:
# Armamos una separacion de los jugadores para analizar diferentes posiciones
forwards=['RF', 'ST', 'LW', 'LF', 'RS', 'LS', 'RM', 'LM','RW']
midfielders=['RCM','LCM','LDM','CAM','CDM','LAM','RDM','CM','RAM','CF']
defenders=['RCB','CB','LCB','LB','RB','RWB','LWB']
goalkeepers=['GK']

def pos2(position):
    if position in forwards:
        return 'Forward'
    
    elif position in midfielders:
        return 'Midfielder'
    
    elif position in defenders:
        return 'Defender'
    
    elif position in goalkeepers:
        return 'GK'
    else:
        return 'nan'

# Cargo en la columna Position2 la clasificacion general de posiciones que armamos con forwards, 
# midfielders, defenders y goalkeepers, utilizando la funcion definida antes pos2
df_fifa22['Position2'] = df_fifa22["player_positions_principal"].str.split(',').str[0].apply(lambda x: pos2(x))


df_fifa22['Position2'].value_counts()

## 4 - Exploratorio Visual para Clustering

Visualizamos los datos (jugadores) según sus habilidades es decir, en un espacio 34 dimensional. Cómo hacemos? empecemos de a dos variables numéricas por vez

**Consigna de trabajo:** visualizar a los jugadores (datos) usando las variables numéricas en skill_ratings de a pares. Emergen grupos/clusters?

In [ ]:
skills_ratings

### Visualizacion con n variables

In [ ]:
# Visualizamos la cantidad de columnas que definimos como parametro de la funcion
sns.pairplot(df_skills[skills_ratings[0:5]]) 

### Visualizacion de dos variables

In [ ]:
# Diferenciamos entre True y False a los mejores jugadores con desempeño por arriba de 85
bool_crack = df_fifa22["overall"] > 85 

##### Ejemplo primer par de variables

In [ ]:
# Elegimos dos variables, para esto elegimos dos números entre 0 y de n_skills-1
skill_1 = skills_ratings[6]
skill_2 = skills_ratings[10]

Gráfica con matplotlib.pyplot, liviana pero más sencilla

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(df_skills[skill_1], y=df_skills[skill_2],s=3,c=bool_crack)
plt.xlabel(skill_1)
plt.ylabel(skill_2)
plt.show()

Gráfica con Plotly, más completa e interactiva pero un poco pesada

In [ ]:
graf1 = go.Scatter(x=df_skills[skill_1], y=df_skills[skill_2],
                           mode='markers',
                           text=df_fifa22.loc[:,'club_name'], #'Height', 'Weight', 'Club', 'Age', 'Name','Position'
                           marker=dict(
                                size=5)
                           )

crack =go.Scatter(x=df_skills.loc[bool_crack,skill_1], y=df_skills.loc[bool_crack,skill_2],name='Top players',
                      text=df_fifa22.loc[bool_crack,'short_name'],
                      textfont=dict(family='sans serif',size=10,color='black'),
                      opacity=0.9,mode='text')

data=[graf1,crack]

layout = go.Layout(title="Visualización de la base de a dos variables numéricas",titlefont=dict(size=20),
                xaxis=dict(title=skill_1),
                yaxis=dict(title=skill_2),
                autosize=False, width=1000,height=650)

fig = go.Figure(data=data, layout=layout)

fig.show()

##### Ejemplo segundo par de variables

In [ ]:
#Elegimos dos variables, para esto elegimos dos números entre 0 y  de n_skills-1
skill_1 = skills_ratings[2]
skill_2 = skills_ratings[3]

Gráfica con matplotlib.pyplot, liviana pero más sencilla

In [ ]:
plt.figure(figsize=(8,8), dpi=80)
plt.scatter(df_skills[skill_1], y=df_skills[skill_2],s=3,c=bool_crack)
plt.xlabel(skill_1)
plt.ylabel(skill_2)
plt.show()

Gráfica con Plotly, más completa e interactiva pero un poco pesada

In [ ]:
graf1 = go.Scatter(x=df_skills[skill_1], y=df_skills[skill_2],
                           mode='markers',
                           text=df_fifa22.loc[:,'club_name'], #'Height', 'Weight', 'Club', 'Age', 'Name','Position'
                           marker=dict(
                                size=5)
                           )

crack =go.Scatter(x=df_skills.loc[bool_crack,skill_1], y=df_skills.loc[bool_crack,skill_2],name='Top players',
                      text=df_fifa22.loc[bool_crack,'short_name'],
                      textfont=dict(family='sans serif',size=10,color='black'),
                      opacity=0.9,mode='text')

data=[graf1,crack]

layout = go.Layout(title="Visualización de la base de a dos variables numéricas",titlefont=dict(size=20),
                xaxis=dict(title=skill_1),
                yaxis=dict(title=skill_2),
                autosize=False, width=1000,height=650)

fig = go.Figure(data=data, layout=layout)

fig.show()

#### Visualizacion por crack

En próxima gráfica, similar a la anterior, también se diferencian los mejores jugadores y alguna característica/variable de interés (que pueden cambiar)

Se puede ubicar también un jugador en particular. Con la siguiente linea de comando, por ejemplo, Leo Messi: 

In [ ]:
#Elegimos un jugador
recherche_joueur=df_fifa22["short_name"]=='L. Messi' 
#recherche_joueur=df_n["Name"]==df_n["Name"][4000] # acá podemos elegir otro 

bool_crack=df_fifa22["overall"] > 85
bool_elecc=df_fifa22["club_name"]=='FC Barcelona' 
bool_no_crack=df_fifa22["overall"]<86
bool_machinegun=df_fifa22["short_name"]=='MachineGun'

#Elegir dos números entre 0 y  de n_skills-1
skill_1=skills_ratings[6]  #6
skill_2=skills_ratings[25]  #25

Gráfica con plotly

In [ ]:
palette=['navy','red','#A2D5F2','orange','green','pink']  

data=[]

n_crack =go.Scatter(x=df_skills.loc[bool_crack,skill_1], y=df_skills.loc[bool_crack,skill_2],name='Crack',
                      text=df_fifa22.loc[bool_crack,'short_name'],
                      textfont=dict(family='sans serif',size=15,color='black'),
                      opacity=0.9,marker=dict(color=palette[2],size=7),mode='markers+text')

n_no_crack =go.Scatter(x=df_skills.loc[bool_no_crack,skill_1], y=df_skills.loc[bool_no_crack,skill_2],name='Average player',
                         text=df_fifa22.loc[bool_no_crack,'short_name'],
                         opacity=0.6,marker=dict(color=palette[1],size=3),mode='markers')
n_elecc=go.Scatter(x=df_skills.loc[bool_elecc,skill_1], y=df_skills.loc[bool_elecc,skill_2],name='Elección',
                         text=df_fifa22.loc[bool_elecc,'short_name'],
                         opacity=0.6,marker=dict(color=palette[0],size=5),mode='markers')

n_machinegun =go.Scatter(x=df_skills.loc[bool_machinegun,skill_1], y=df_skills.loc[bool_machinegun,skill_2],name='Perfect player',
                           textfont=dict(family='sans serif',size=20,color='black'),
                           opacity=0.6,marker=dict(color=palette[3],size=30),mode='markers+text')


joueur_recherche =go.Scatter(x=df_skills.loc[recherche_joueur,skill_1], y=df_skills.loc[recherche_joueur,skill_2],name='Searched player',
                           text=df_fifa22.loc[recherche_joueur,'short_name'],
                            textfont=dict(family='sans serif',size=20,color='black'),
                           opacity=1,marker=dict(color=palette[4],size=40),mode='markers+text')

data=[n_no_crack,n_elecc
      ,n_crack,n_machinegun,joueur_recherche]

layout = go.Layout(title="Fifa Players",titlefont=dict(size=20),
                xaxis=dict(title=skill_1),
                yaxis=dict(title=skill_2),
                autosize=False, width=1000,height=650)

fig = go.Figure(data=data, layout=layout)


In [ ]:
#En Colab
fig.show()

**Identifiquemos algunos jugadores por nombre**

In [ ]:
print(df_fifa22.loc[0:43,"short_name"])

# EJERCICIO 3 - Uso de dos técnicas de clustering

##**Clustering con Kmeans**

#### Analisis de Codo

El objetivo es determinar la cantidad de clusters que el metodo propone como cantidad optima

In [ ]:
# Analisis para determinar el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i).fit(df_skills).inertia_ for i in range(2,12)]

plt.figure(figsize=(12,9))
plt.plot(np.arange(2, 12), scores, marker='o', linestyle='--')
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("Inertia of k-Means versus number of clusters")

#### Aplicamos KMeans

In [ ]:
# Número de clusters definido por el metodo del codo
clusters =4

# Creo una variable con el metodo KMeans
km = KMeans(n_clusters=clusters)
# Enterno mi modelo utilizando 34 dimensiones que forman las variables definidas como skills
km.fit(df_skills)

In [ ]:
# Diferentes clusters asignadas a cada jugador del data frame df_skills por el algoritmo que entrenamos 
clusters = km.labels_

print('Kmeans encontró: ', max(km.labels_)+1, 'clusters')

In [ ]:
# Copio la base de analisis df_fifa22 a la df_KMeans y le agrego la columna con el cluster generados a partir de entrenar el modelo con 
# las 34 instancias que forman parte de df_skill 
df_KMeans=df_fifa22.copy()
df_KMeans['KMeans'] = clusters
df_KMeans

#### Analisis del Data Frame

##### Analisis General

In [ ]:
# Cantidad de registros asociados a cada clusters
df_KMeans['KMeans'].value_counts()

In [ ]:
# Corremos silhouette para saber que tan emparejados estan los elementos de un mismo grupo y que tan diferentes son del os otros grupos
silhouette_score = metrics.silhouette_score(df_skills, clusters, metric='euclidean')
silhouette_score

In [ ]:
# Para que no me genere una distoncion de los grafico y tablas de analisis eliminamos de KMeans el MacineGun
df_KMeans.drop(df_KMeans.loc[df_KMeans['short_name']=='MachineGun'].index, inplace=True)

# Armo una crosstable con el porcentaje de los registros asociados a cada cluster creado por KMeans
cross = pd.crosstab(df_KMeans.KMeans, df_KMeans.Position2, normalize="index")
cross

In [ ]:
# Armo un grafico para saber porcentualmente como se distribuyen los registros de las cuatro posiciones en los distintos cluster
plt.figure(figsize = (15,8))
sns.heatmap(cross, annot=True, cmap = "OrRd")

##### Variables de Analisis

In [ ]:
# Eligo dos dos variables de los skills definiendo numeros entre 0 y los n_skills
skill_1=skills_ratings[9]
skill_2=skills_ratings[10]

##### Cracks

In [ ]:
# Defino dos grupos booleanos para agregar los nombres en la grafica separando dos grupos de jugadores
bool_crack = df_fifa22["overall"] > 85
bool_crack.value_counts()


In [ ]:
# 1° Defino un grupo del data frame df_skills con la primer variable en funcion de la segunda y 
# los coloreo en funcion del cluster asignado por KMean 
kmean_clusters = go.Scatter(x=df_skills[skill_1], y=df_skills[skill_2], mode='markers', text=df_KMeans.loc[:,'short_name'],
                           marker=dict(size=5, color=clusters.astype(np.float), #set color equal to a variable
                                colorscale='Portland', showscale=False))

# 2° Defino el grupo de de jugadores que forman el grupo de bool_crack considerando 
# la primer variable en funcion de la segunda. Se arma este grupo selecto para que aparezcan los nombres 
# pero los valores on iguales a los presentados en el grafico anteriror
crack =go.Scatter(x=df_skills.loc[bool_crack,skill_1], y=df_skills.loc[bool_crack,skill_2], name='Nombres de Ckacks',
                      text=df_KMeans.loc[bool_crack,'short_name'],
                      textfont=dict(family='sans serif',size=10,color='black'),
                      opacity=0.9,mode='text')

data=[kmean_clusters,crack]

layout = go.Layout(title="Clustering K means para Cracks!!!",titlefont=dict(size=20),
                xaxis=dict(title=skill_1),
                yaxis=dict(title=skill_2),
                autosize=False, width=1000,height=650)

fig = go.Figure(data=data, layout=layout)

In [ ]:
fig.show()

##### Mediocres

In [ ]:
# Defino dos grupos booleanos para agregar los nombres en la grafica separando dos grupos de jugadores
# Vemos que tenemos muchos nombre para imprimir en la grafica con lo cual puede salir una mancha que tape los clusters
bool_no_crack = df_fifa22["overall"] <= 71
bool_no_crack.value_counts()

In [ ]:
# 1° Defino un grupo del data frame df_skills con la primer variable en funcion de la segunda y 
# los coloreo en funcion del cluster asignado por KMean 
kmean_clusters = go.Scatter(x=df_skills[skill_1], y=df_skills[skill_2], mode='markers', text=df_KMeans.loc[:,'short_name'],
                           marker=dict(size=5, color=clusters.astype(np.float), #set color equal to a variable
                                colorscale='Portland', showscale=False))

# 2° Defino el grupo de de jugadores que forman el grupo de bool_crack considerando 
# la primer variable en funcion de la segunda. Se arma este grupo selecto para que aparezcan los nombres 
# pero los valores on iguales a los presentados en el grafico anteriror
no_crack =go.Scatter(x=df_skills.loc[bool_no_crack,skill_1], y=df_skills.loc[bool_no_crack,skill_2], name='Nombres Regulares',
                     # text=df_KMeans.loc[bool_no_crack,'short_name'],
                      textfont=dict(family='sans serif',size=10,color='black'),
                      opacity=0.9,mode='text')

data=[kmean_clusters,no_crack]

layout = go.Layout(title="Clustering K means para Regulares!!!",titlefont=dict(size=20),
                xaxis=dict(title=skill_1),
                yaxis=dict(title=skill_2),
                autosize=False, width=1000,height=650)

fig = go.Figure(data=data, layout=layout)

In [ ]:
fig.show()

##**Clustering con DBSCAN**

#### Analisis de Rodilla o Epsilon

In [ ]:
# Definimos las dos variables que vamos a analisar que las sacamos del df_fifa22 y son parte del skills_ratings
df_knn = df_fifa22.copy()[['movement_acceleration', 'skill_ball_control']]
#data_nn = df_fifa22.copy()[['movement_acceleration', 'skill_ball_control']]

In [ ]:
# Utilizo el metodo de Vecinos mas cercanos KNN para 4 vecinos defino la variable 
knn = NearestNeighbors(n_neighbors=4)
vecinos = knn.fit(df_knn)

# Saco la matriz de distancias de cada punto analizado (filas) respecto cada uno 
# de los vecinos mas cercanos (columnas) 
distancia, indices = vecinos.kneighbors(df_knn)

# Ordeno por filas las 4593 filas que forman la matriz del df_knn reducido del a dos variables
distancia = np.sort(distancia, axis=0)
print('Cantidad de puntos analisados que forman las dos varibales seleccionadas:',len(distancia))
print('\nMatriz de distancia de cada punto con respecto a los vecinos seleccionados\n', distancia)


In [ ]:
# De la matriz que calculamos anteriormente tomamos una columna para hacer el analisis del mejor EPSILON
columna=3
distancia_vector = distancia[:,columna]
print('Datos de la columna de distancias seleccionada de la matriz\n', distancia_vector)

# Me armo el vector de elementos que toma todas las filas incluidas en la definicion de distancias
i = np.arange(len(distancia_vector))
print('\nCantidad de puntos analisados que forman las dos varibales seleccionadas\n', i)

In [ ]:
# Determinamos el EPSILON correcto en funcion de los dos vectores que armamos antes
# PARA DETERMINAR EL EPSILON CORRECTO TENEMOS QUE ANALIZAR EL MAYOR QUIEBRE QUE SE PRODUCE EN LA GRAFICA 


plt.figure(figsize=(12,9))
sns.lineplot(x=i, y=distancia_vector)
plt.xlabel("Punto")
plt.ylabel("Distancia")

Podemos determinar que utilizando cualquier columna de las distancias que se generan por cada punto con respecto, en este caso a los cuatro vecinos mas cercanos, el EPSILON que se grafica con el quibre mas pronunciado seria 1 

#### Aplicamos DBScans

In [ ]:
# Creo una variable con el metodo DBScan y utilizo el Epsilon que calculamos antes
dbs = DBSCAN(eps=1, min_samples=30)

# Realizamos una copia de la base de datos original
df_dbscan = df_fifa22.copy()

# Enterno mi modelo utilizando solo 2 dimensiones represntadas por las variables definidas como skills
df_dbscan['dbscan'] = dbs.fit_predict(df_dbscan[['movement_acceleration', 'skill_ball_control']])
df_dbscan

#### Analisis de Data Frame

In [ ]:
# Cantidad de clusters creados por el metodo DBScan
df_dbscan['dbscan'].unique()

In [ ]:
# Cantidad de registros asociados a cada clusters
df_dbscan['dbscan'].value_counts()

In [ ]:
# Corremos silhouette 
silhouette_score = metrics.silhouette_score(df_dbscan[['movement_acceleration', 'skill_ball_control']], clusters, metric='euclidean')
silhouette_score

Comentario del resultado del la metrica

In [ ]:
# Para que no me genere una distoncion de los grafico y tablas de analisis eliminamos de KMeans el MacineGun
df_dbscan.drop(df_dbscan.loc[df_dbscan['short_name']=='MachineGun'].index, inplace=True)

# Armo una crosstable con los registros asociados a cada cluster creado
cross = pd.crosstab(df_dbscan.dbscan, df_dbscan.Position2, normalize="index")
cross

In [ ]:
# Armo un grafico para saber porcentualmente como se distribuyen los registros de las cuatro posiciones en los distintos cluster
plt.figure(figsize = (15,8))
sns.heatmap(cross, annot=True, cmap = "OrRd")

In [ ]:
# Generamos el grafico pasando por parametro los datos c que genero el DBScan

plt.figure(figsize=(12,9))
plt.scatter(df_dbscan['movement_acceleration'], df_dbscan['skill_ball_control'], c=df_dbscan['dbscan'])

In [ ]:
# Defino una lista numerica para agregar una columna nueva con valores numericos que identifican la calsificacion anterior
posicion_numerica = {'Defender' : 0, 'Forward' : 1, 'GK' : 2, 'Midfielder' : 3}

# Copio la columna Position2 en Position3 y la trasnformo en numerico
df_dbscan['Position3'] = df_dbscan['Position2']
df_dbscan.replace({"Position3": posicion_numerica}, inplace=True)
df_dbscan['Position3']

In [ ]:
# Grafico por las cuatro posiciones definidas 'Defender' : 0, 'Forward' : 1, 'GK' : 2, 'Midfielder' : 3

plt.figure(figsize=(12,9))
plt.scatter(df_dbscan['movement_acceleration'], df_dbscan['skill_ball_control'], c=df_dbscan['Position3'])


In [ ]:
# Grafico coloreando diferente a las posiciones 'Defender' : 0, 'Forward' : 1, 'GK' : 2, 'Midfielder' : 3

plt.figure(figsize=(12,9))
plt.scatter(df_dbscan['movement_acceleration'], df_dbscan['skill_ball_control'],
            #c = df_dbscan['Position3']!=0,  # Marcamos en otro color los Defensores
            #c = df_dbscan['Position3']!=1,  # Marcamos en otro color los Delanteros
            c = df_dbscan['Position3']!=2,  # Marcamos en otro color los Arqueros
            #c = df_dbscan['Position3']!=3,  # Marcamos en otro color los Mediocampistas
            cmap = 'Set1' #'inferno' 'viridis'
)

# EJERCICIO 4 - Evaluación y Análisis de los clusters encontrados.

Antes de evaluara y analizar el resultados de los clusters es importante aclarar que se redujo el data frame original a 4600 elementos y se consideraron 34 variables o instancias de analisis. 
En la determinacion de los Clusters se utilizaron dos metodos de analisis 
*   **KMEANS** para lo cual se realizo el Analisis de Codo y se determino que la cantidad conveniente de clusters para el analisis era 4. Con esta cantidad se entreno en modelo con el dataset con los mejores jugadores  considerando las 34 variables de skill. Para su analisis visual su utilizaron  dos variables/instancias aleatorias. En este caso analizando el mapa de calor que surge delos clusters generados se puede verificar que:
  1.   **Cluster 0** contiene el 100% de los arqueros
  2.   **Cluster 1** es mas variado pero contine 54% de Mediocampistas y 38% de defensores que seguramente son lo que se proyectan al mediocampo
  3.   **Cluster 2** es mas marcada la formacion con jugadores Delanteros en un 80% y solo 20% Mediocampistas que seguramente son lo que se proyectan al ataque.
  4.   **Cluster 3** esta conformado en un 89% de defensores y 11% de Mediocampistas que seguramente tienen skill propensos a realizar tareas de defensa.

*   **DBSCAN** realiza el analizis de clusterizacion con este metodo tomamos dos variables/instancias "movement_acceleration" y "skill_ball_control". Para el entrenamiento de este modelos tenemos fijar el valor de dos parametros: 
  1.   El parametro EPSILON para su estimacion optima se utilizo el KNN con la cantidad de puntos centroides a partir de los cuales definimos las distancias desde cada punto de la base a dichos centrides. Determinadas estas distancias se calculo el optimo EPSILON utilizando el metodo de Rodilla cuyo analisis de distancias dio como resultado el valor 1.  
  2.   El min_samples que determina la cantidad de puntos que DBSCAN va a utilizar para armar un cluster.

El metodo genera 10 cluster en su proceso, con el grafico de calor podemos visualizar que los Defensores en porcentaje superior al 50% se encuentar en los cluster 1, 3, 5 y 8, para los Arqueros se encuentran totalmente contenidos en el cluster -1, los Medicampistas considerando en porcentaje mayor al 50% se encuentras en los clusters 4, 6 y 7. Para el caso de los Delanteros estan distribuidos proporcionalmente en todos los clusters pero se presentan casi 40%, en los clusters 0 y 2. Por lo tanto se puede estimar que los clusters estan representando los 4 diferentes agrupamientos de posiciones 

#Ejercicio 5 - Pregunta: ¿Se realizó alguna normalización o escalado de la base? ¿Por qué ?

No fue necesaria realizar ninguna normalizacion porque todas, las 34 variables/intancias que se tomaron en cuenta para el analisis y que forman parte del skill de jugador varian entre 0 y 100

#Ejercicio 6 - Uso de alguna transformación (proyección, Embedding) para visualizar los resultados y/o usarla como preprocesado para aplicar alguna técnica de clustering.

## Aplico Embedding con PCA

### Defino Ambiente

Como empezamos a realizar un analisis detallado de la base tomamos el total de los registrospara realizar la transformacion

In [ ]:
# Realizo una copia del data frame para empezar el analisis
train = df.copy()

In [ ]:
# Tomo el data frame original completo y me quedo con las columnas solo numericas
train = train.select_dtypes(['number'])
train.isnull().sum()

In [ ]:
# El metodo PCA no trabaja con valores nulos porque da error por lo tanto antes de aplicarlo tenemos que eliminarlos

# Elimino aquellas columnas que tienen muchos valores nulos en este caso por arriba de los 4000 registros y que no impactan al analisis que vamos a realizar
train = train.drop(['goalkeeping_speed', 'nation_team_id', 'nation_jersey_number'], axis=1)

# Elimino todos los registros que tienen algun valor nulo
train = train.dropna()
train.info()

### Estandarizo el Data Frame

In [ ]:
# Como este metodo genera un arreglo de valores, me salvo las columnas para 
# cargarlas de nuevo en el data frame resultado
x_names = train.columns
array_standard = StandardScaler().fit_transform(train)
# Paso el resultado de la estandarizacion al data frame que voy a usar
df_train = pd.DataFrame(array_standard, columns=x_names)
df_train

### Proceso PCA

In [ ]:
# Creo ls variable PCA, entreno y transformo el modelo genero el resultado 
pca = PCA(n_components=3)
scores_pca = pca.fit_transform(df_train)
scores_pca

In [ ]:
# Grafico la distribucion del calculo generada por el PCA
plt.figure(figsize=(12,9))
sns.scatterplot(x = scores_pca[:,0], y = scores_pca[:,1], palette = sns.hls_palette(10), legend = 'full');

In [ ]:
# Preparo mi data frame para la visualizacion entonces realizo la la union del data frame sin estandarizar
# train con los valores que se obtubieron de PCA
df_segm_pca= pd.concat([train.reset_index(drop=True), pd.DataFrame(scores_pca)], axis=1)

# A esas nuevas columnas agregadas le asigno un nombre
df_segm_pca.columns.values[-3: ]= ['Component 1','Component 2','Component 3']

df_segm_pca

UTILIZAR EL DATA FRAME DF_SEGM_PCA 
TIENA ASIGNADAS LAS COLUMNAS RESULTADO DEL PCA

### Procesos de Clustering

#### KMEANS

##### Analisis de Codo

In [ ]:
# Analisis para determinar el hiperparámetro n_clusters, variando de 2 a 11 clusters
scores = [KMeans(n_clusters=i, init='k-means++', random_state=42).fit(scores_pca).inertia_ for i in range(2,12)]

plt.figure(figsize=(12,9))
plt.plot(np.arange(2, 12), scores, marker='o', linestyle='--')
plt.xlabel('Number of clusters')
plt.ylabel("Inertia")
plt.title("K-Means  with PCA")

##### Clustering KMeans Optimo

COMO KMEAN UTILIZA LA TOTALIDAD DE INSTANCIAS QUE FORMAN EL DATA SET DE ANALISIS, CONSIDERAMOS PARA LA DEFINICION DE CLUSTERS SOLAMENTE EL ARRAY RESULTADO DEL DATA FRAME AL CUAL SE APLICO **PCA**

In [ ]:
# Clustering K-Means sobre el array score_pca que obtubimos resultado de 
# aplicar PCA en data frame estandarizado sin nulos df_train
kmeans_pca = KMeans(n_clusters= 4, init='k-means++', random_state=42)
kmeans_pca.fit(scores_pca)

In [ ]:
# Del tata frame que armamos con los resultados del PCS vamos a realizar una copia para trabajar los clusters de KMean  
df_segm_pca_kmeans = df_segm_pca.copy()

# Al data frame que aramamos con los resultados de PCA le agrego la columna de los clusters que sale del analisis de KMeans
df_segm_pca_kmeans['Segment k-means PCA']= kmeans_pca.labels_

df_segm_pca_kmeans

In [ ]:
# Media de todas la svariables numericas considerando la segmentacion que genero
df_segm_pca_kmeans_freq= df_segm_pca_kmeans.groupby(['Segment k-means PCA']).mean()
df_segm_pca_kmeans_freq

In [ ]:
plt.figure(figsize=(12,9))
x_axis=df_segm_pca_kmeans['Component 1']
y_axis=df_segm_pca_kmeans['Component 2']
sns.scatterplot(x_axis, y_axis, hue= df_segm_pca_kmeans['Segment k-means PCA'], palette=['g','r','c','m'])
plt.title('Clusters por componentes PCA')
plt.show()

##### Clustering KMeans con n Clusters

COMO KMEAN UTILIZA LA TOTALIDAD DE INSTANCIAS QUE FORMAN EL DATA SET DE ANALISIS, CONSIDERAMOS PARA LA DEFINICION DE CLUSTERS SOLAMENTE EL ARRAY RESULTADO DEL DATA FRAME AL CUAL SE APLICO **PCA**

In [ ]:
# Clustering K-Means sobre el array score_pca que obtubimos resultado de 
# aplicar PCA en data frame estandarizado sin nulos df_train
kmeans_pca = KMeans(n_clusters= 20, init='k-means++', random_state=42)
kmeans_pca.fit(scores_pca)

In [ ]:
# Del tata frame que armamos con los resultados del PCS vamos a realizar una copia para trabajar los clusters de KMean  
df_segm_pca_kmeans = df_segm_pca.copy()

# Al data frame que aramamos con los resultados de PCA le agrego la columna de los clusters que sale del analisis de KMeans
df_segm_pca_kmeans['Segment k-means PCA']= kmeans_pca.labels_

df_segm_pca_kmeans

###### Visualizacion KMeans n Clusters

In [ ]:
# Media de todas la svariables numericas considerando la segmentacion que genero el PCA
df_segm_pca_kmeans_freq= df_segm_pca_kmeans.groupby(['Segment k-means PCA']).mean()
df_segm_pca_kmeans_freq

In [ ]:
plt.figure(figsize=(12,9))
x_axis=df_segm_pca_kmeans['Component 2']
y_axis=df_segm_pca_kmeans['Component 3']
sns.scatterplot(x_axis, y_axis, hue= df_segm_pca_kmeans['Segment k-means PCA'])
plt.title('Clusters por componentes PCA')
plt.show()

###### Visualizacion KMeans solo 4 de los 20 Clusters

In [ ]:
df_segm_pca_kmeans['Legend']= df_segm_pca_kmeans['Segment k-means PCA'].map({0:'Primer Cluster',1:'Segundo Cluster',2: 'Tercer Cluster',3:'Cuarto Cluster'})

plt.figure(figsize=(12,9))
x_axis=df_segm_pca_kmeans['Component 1']
y_axis=df_segm_pca_kmeans['Component 2']
sns.scatterplot(x_axis, y_axis, hue= df_segm_pca_kmeans['Legend'])

#### MEZCLA GAUSSIANA

##### Analisis Parametros

In [ ]:
#prueba con mezcla de gaussianas con PCA. Su procesamiento lleva tiempo

bic = []
aic = []
for i in range(8):
    gm = GaussianMixture(n_components = i+1, n_init = 10, max_iter = 100)
    gm.fit(df_segm_pca)
    bic.append(gm.bic(df_segm_pca))
    aic.append(gm.aic(df_segm_pca))

fig = plt.figure()
plt.plot([1,2,3,4,5,6,7,8], aic)
plt.plot([1,2,3,4,5,6,7,8], bic)
plt.show()

##### Clustering con MG

In [ ]:
gm_pca = GaussianMixture(n_components = 5, random_state =42) 
gm_pca.fit(df_segm_pca)
predicted_values = gm_pca.predict(df_segm_pca)

In [ ]:
# Del tata frame que armamos con los resultados del PCA vamos a realizar una copia para trabajar los clusters de MG  
df_segm_pca_gm = df_segm_pca.copy()

# Al data frame que aramamos con los resultados de PCA le agrego la columna de los clusters que sale del analisis de MG
df_segm_pca_gm['Segment GM PCA']= predicted_values

df_segm_pca_gm

In [ ]:
# Cantidad de registros asociados a cada clusters
df_segm_pca_gm['Segment GM PCA'].value_counts()

In [ ]:
plt.figure(figsize=(12,9))
x_axis=df_segm_pca_gm['Component 1']
y_axis=df_segm_pca_gm['Component 2']
sns.scatterplot(x_axis, y_axis, hue= df_segm_pca_gm['Segment GM PCA'])
plt.title('Clusters por componentes PCA')
plt.show()